# OPR Prototype 

To standardize output from People Insight - Operating Performance Report and to put in Pay Band and other Reward information for the report.

In [ ]:
import pandas as pd
import numpy as np

### Pay Band Master

In [ ]:
# Importing the GBR and ROI Pay Band Masters
PBM = pd.read_csv('UKI PBM.csv', encoding='latin1')
ROI = pd.read_csv('ROI PBM.csv', encoding='latin1')

In [ ]:
# Removing the default UJI [000, 010, 020] from the 2 Pay Band Masters
default=["000","010","020"]

PBM = PBM.set_index("UJI")
PBM.drop(default, axis=0, inplace=True)
PBM = PBM.reset_index(inplace = False)

ROI = ROI.set_index("UJI")
ROI.drop(default, axis=0, inplace=True)
ROI = ROI.reset_index(inplace = False)

In [ ]:
# Selecting columns required for merge
col_merge = ['UJI', 'UJI Level', 'Location', 'Pay Band', 'Payband Level', 'UJI Library Short Description', 'Entry', 'Mid Point', 'Top', 'Market Median']

PBM = PBM[col_merge]
ROI = ROI[col_merge]

In [ ]:
# Combine the two Pay Band Masters into one reference
PBM = pd.concat([PBM, ROI], ignore_index=True)

In [ ]:
# Formatting the mid point column
PBM['Mid Point'] = PBM['Mid Point'].astype(str).str.replace(',', '')
PBM['Mid Point'] = PBM['Mid Point'].astype(str).str.replace(' ', '').astype(float)

# Removing duplicates based on UJI and Location
PBM = PBM.sort_values("Mid Point", ascending=False)
PBM = PBM.drop_duplicates(["UJI","Location"])

#Sort by UJI
PBM = PBM.sort_values("UJI")

### OPR

In [ ]:
# Importing the OPR
OPR = pd.read_csv('Operational People Roster.csv')

In [ ]:
# Calculate FTE Salary
OPR['FTE Salary'] = OPR['AnnualCompensationRate'].astype(str).str.replace(',', '').astype(float)/(OPR['FTE1'])
OPR['FTE Salary'] = OPR['FTE Salary'].replace(np.inf, np.nan)

In [ ]:
# Combine with pay band information
OPR = pd.merge(OPR, PBM, how='left', left_on= ['SalaryGrade','LocationCity'], right_on= ['UJI', 'Location'])

### Pay Band Information

In [ ]:
# Formatting the columns
OPR['Entry'] = OPR['Entry'].astype(str).str.replace(',', '')
OPR['Entry'] = OPR['Entry'].astype(str).str.replace(' ', '').astype(float)

OPR['Top'] = OPR['Top'].astype(str).str.replace(',', '')
OPR['Top'] = OPR['Top'].astype(str).str.replace(' ', '').astype(float)

OPR['AnnualCompensationRate'] = OPR['AnnualCompensationRate'].astype(str).str.replace(',', '')
OPR['AnnualCompensationRate'] = OPR['AnnualCompensationRate'].astype(str).str.replace(' ', '').astype(float)

OPR['Mid Point'] = OPR['Mid Point'].astype(str).str.replace(',', '')
OPR['Mid Point'] = OPR['Mid Point'].astype(str).str.replace(' ', '').astype(float)

In [ ]:
# Calculating Compa-Ratio
OPR['Compa-Ratio'] = np.where(OPR['FTE Salary']==0, np.nan, OPR["FTE Salary"]/OPR["Mid Point"])
OPR['Compa-Ratio'] = OPR['Compa-Ratio'].replace(np.inf, np.nan)

In [ ]:
# Deriving entity basing on Business Unit
OPR["Entity"] = np.where(OPR['BU'].str.startswith("IE"), "ROI", "UKI")
OPR['Entity'] = np.where(OPR['BU'].str.startswith("GB080"), "FSO", OPR.Entity)
ROIFS = ["IE002","IE032", "IE042", "IE062"]
OPR['Entity'] = np.where(OPR['BU'].isin(ROIFS), "ROI FS", OPR.Entity)


EY levels derived from Rank

In [ ]:
Manager=[32, 63]
Director=[21, 62, 13, 61]
AM=[64,55]
# 42 Grade 4 ONLY
Others=[44,57,58,66,65,55, 42]
IE=["ROI FS", "ROI"]
UK=["FSO", "UKI"]

In [ ]:
# Calculating tenure
from datetime import datetime
OPR['CurrentEmploymentDate1']= pd.to_datetime(OPR['CurrentEmploymentDate1'], format='%d-%b-%y')
# Tenure calculated based on today's date - should potentially change to an input box for back dated reports
now = pd.Timestamp('now')
OPR['Tenure'] = OPR['CurrentEmploymentDate1'].where(OPR['CurrentEmploymentDate1'] < now, OPR['CurrentEmploymentDate1'] -  np.timedelta64(100, 'Y'))
OPR['Tenure']= (now - OPR['Tenure']).astype('<m8[Y]')
OPR['CurrentEmploymentDate1'] = OPR['CurrentEmploymentDate1'].dt.strftime('%d-%b-%y')

In [ ]:
# Not sure why OPR2 was created
OPR2 = OPR.copy()
OPR2 = OPR2[['GPN', 'LPN', 'LastName', 'FirstName', 'JobCode', 'JobCodeDesc1', 'SalaryGrade','UJI Level','UJI Library Short Description','Pay Band', 'Rank','RankDesc','EYGrade','BusinessTitleJobPage','JobFamily','JobFamilyDesc','FTE Salary', 'AnnualCompensationRate','Entry', 'Mid Point', 'Top', 'Market Median', 'Compa-Ratio','Entity','BU','BUName','ServiceLine','SubSL1','DepartmentCode1','DepartmentName1','OU','OUName','MU','MUName','SMU','SMUName','Codeblock','LocationCode','LocationCity','LocationAddress1','StandardHours','StandardHoursSalaryAdminPlan','FTE1','EmployeeClass','RegularTemporary','PaygroupCode','EmployeeCategoryCode','EmployeeCategory','CurrentEmploymentDate1','SeniorityPayDate','GUI','EmployeeID1','EmployeeStatus1','CounselorGUI','CounselorName','gTESupervisorGUI','gTESupervisorName','EmailAddress','GenderCode','Ethnicity','MaritalStatus','DateOfBirth','EmployeeRecord', 'Tenure']]
OPR2.rename(columns = {'SalaryGrade': 'UJI', 'UJI Library Short Description': 'UJI Description','BusinessTitleJobPage': 'Job Title', 'AnnualCompensationRate': 'Actual Salary', 'Mid Point': 'Mid', 'GenderCode': 'Gender'}, inplace = True)

#### Allowances
Providing information on PMI, car allowance and pension.

In [ ]:
# PMI calculations
OPR2['PMI'] = np.where((OPR2['Rank'].isin(Manager)) | (OPR2['Rank'].isin(Director)), 1092.72, np.nan)
OPR2['PMI'] = np.where(OPR2['Tenure'] > 4, 1092.72, OPR2['PMI'])
OPR2['PMI Description'] = np.where((OPR2['Rank'].isin(Manager)) | (OPR2['Rank'].isin(Director)), "Single Cover", "")
OPR2['PMI Description'] = np.where(OPR2['Tenure'] > 4, "Single Cover", OPR2['PMI Description'])

#Finetuning the categories in ROI
OPR2['PMI Description'] = np.where(OPR2['Entity'].isin(IE), np.nan, OPR2['PMI Description'])
OPR2['PMI Description'] = np.where((OPR2['Rank'].isin(Manager)) & (OPR2['Entity'].isin(IE)) |(OPR2['Rank'].isin(AM)) & (OPR2['Entity'].isin(IE)) | (OPR2['Rank'].isin(Director)) & (OPR2['Entity'].isin(IE)), "Medical Allowance", OPR2['PMI Description'])
OPR2['PMI Description'] = np.where((OPR2['Rank']==42) & (OPR2['Entity'].isin(IE)) & (OPR2['EYGrade']==4), "Medical Allowance", OPR2['PMI Description'])
OPR2['PMI'] = np.where(OPR2['Entity'].isin(IE), np.nan, OPR2['PMI'])
OPR2['PMI'] = np.where((OPR2['Rank'].isin(Manager)) & (OPR2['Entity'].isin(IE)) |(OPR2['Rank'].isin(AM)) & (OPR2['Entity'].isin(IE)) | (OPR2['Rank'].isin(Director)) & (OPR2['Entity'].isin(IE)), 660, OPR2['PMI'])
OPR2['PMI'] = np.where((OPR2['Rank']==42) & (OPR2['Entity'].isin(IE)) & (OPR2['EYGrade']==4), 660, OPR2['PMI'])

# Car allowance calculations
OPR2['Car'] = np.where(OPR2['Rank'].isin(Manager), 4500, np.nan)
OPR2['Car'] = np.where(OPR2['Rank'].isin(Director), 5500, OPR2['Car'])
OPR2['Car'] = np.where(OPR2['Entity'].isin(IE), np.nan, OPR2['Car'])

# Pensions calcuations
OPR2['FTE Pension'] = np.where(OPR2['Entity'].isin(IE), 0.07, 0.06)
#OPR2['FTE Pension'] = np.where(OPR2['BU'].str.startswith("IE"), OPR2["FTE Salary"]* 0.07, OPR2["FTE Salary"]* 0.06 * 1.138)

#Allowance = OPR2[["FTE Pension", "Car", "PMI"]] 
#OPR2["FTE Allowance"] = Allowance.sum(axis=1)


In [ ]:
Output = OPR2.copy()

### Drop/Rename Columns

In [ ]:
Output = Output[['GPN', 'LPN', 'LastName', 'FirstName', 'JobCode', 'JobCodeDesc1', 'UJI','UJI Level','UJI Description', 'Pay Band', 'Rank','RankDesc','EYGrade','Job Title','JobFamily','JobFamilyDesc','FTE Salary', 'Actual Salary','PMI','PMI Description', 'Car', 'FTE Pension', 'Entry', 'Mid', 'Top', 'Market Median', 'Compa-Ratio','Entity','BU','BUName','ServiceLine','SubSL1','DepartmentCode1','DepartmentName1','OU','OUName','MU','MUName','SMU','SMUName','Codeblock','LocationCode','LocationCity','LocationAddress1','StandardHours','StandardHoursSalaryAdminPlan','FTE1','EmployeeClass','RegularTemporary','PaygroupCode','EmployeeCategoryCode','EmployeeCategory','CurrentEmploymentDate1','SeniorityPayDate','GUI','EmployeeID1','EmployeeStatus1','CounselorGUI','CounselorName','gTESupervisorGUI','gTESupervisorName','EmailAddress','Gender','Ethnicity','MaritalStatus','DateOfBirth']]

### VPS

In [ ]:
# Importing Incident Earnings output from People Insight
VPS = pd.read_csv('Incidental Earnings - removed.csv', encoding='latin1')

In [ ]:
Output = pd.merge(Output, VPS, how='left', left_on= ['GPN'], right_on= ['GPN'])

In [ ]:
Output['Position in Band'] = np.where(Output['FTE Salary']==0, '', np.where(pd.isna(Output['Mid']), '', np.where(Output['FTE Salary']<Output['Entry'], 'Below Entry', np.where(Output['FTE Salary']<((Output['Entry']+Output['Mid'])/2), 'Q1', np.where(Output['FTE Salary']<Output['Mid'], 'Q2', np.where(Output['FTE Salary']<((Output['Top']+Output['Mid'])/2), 'Q3', np.where(Output['FTE Salary']<=Output['Top'], 'Q4', 'Exceed Top'))))))) 

### Salary Survey Match

In [ ]:
submissions = pd.read_csv('2019 Submissions.csv', encoding='latin1')

In [ ]:
Output = pd.merge(Output, submissions, how='left', left_on= ['GPN'], right_on= ['GPN'])

In [ ]:
Output = Output[['GPN', 'LPN', 'LastName_x', 'FirstName_x', 'JobCode', 'JobCodeDesc1', 'UJI','UJI Level','UJI Description', 'Pay Band', 'Rank','RankDesc','EYGrade','Job Title','JobFamily','JobFamilyDesc','FTE Salary', 'Actual Salary','PMI','PMI Description', 'Car', 'FTE Pension', 'Entry', 'Mid', 'Top', 'Position in Band', 'Compa-Ratio','Entity','BU','BUName','ServiceLine','SubSL1','DepartmentCode1','DepartmentName1','OU','OUName','MU','MUName','SMU','SMUName','Codeblock','LocationCode','LocationCity','LocationAddress1','StandardHours','StandardHoursSalaryAdminPlan','FTE1','EmployeeClass','RegularTemporary','PaygroupCode','EmployeeCategoryCode','EmployeeCategory','CurrentEmploymentDate1','SeniorityPayDate','GUI','EmployeeID1','EmployeeStatus1','CounselorGUI','CounselorName','gTESupervisorGUI','gTESupervisorName','EmailAddress','Gender','Ethnicity','MaritalStatus','DateOfBirth', 'Amount1', 'WTW GI','McLagan','KEYPAD','New KEYPAD','Actuarial','Act level','Comp Res','Comp level'
]]
Output.rename(columns = {'LastName_x':'LastName', 'FirstName_x':'FirstName','Amount1': 'VPS'}, inplace = True)

### Exporting

In [ ]:
# Saving as .csv file
Output.to_csv(r'C:\Users\2022464\Downloads\Learning\Python\OPR\Output.csv',encoding='utf-8-sig', index=False)